In [1]:
import pandas as pd
import cobra
import copy
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

ListOfRandomSpecies = [
'Bacteroides_thetaiotaomicron_VPI_5482',
'Bacteroides_sp_2_2_4',
'Parabacteroides_johnsonii_DSM_18315',
'Prevotella_oralis_ATCC_33269',
'Eubacterium_eligens_ATCC_27750',
'Slackia_exigua_ATCC_700122',
'Dorea_longicatena_DSM_13814',
'Clostridium_bartlettii_DSM_16795',
'Streptococcus_sp_I_P16',
'Blautia_hydrogenotrophica_DSM_10507',
'Brevundimonas_bacteroides_DSM_4726',
'Clostridium_hylemonae_DSM_15053',
'Sutterella_wadsworthensis_3_1_45B',
'Enterobacteriaceae_bacterium_9_2_54FAA',
'Bacillus_megaterium_DSM319',
#'Peptostreptococcus_stomatis_DSM_17678', # average-european-diet causes problems
'Brachybacterium_paraconglomeratum_LC44',
'Neisseria_elongata_subsp_glycolytica_ATCC_29315',
'Rothia_aeria_F0474',
'Staphylococcus_hominis_subsp_hominis_C80', 
]

betternames = [n.split('_')[0][0] + '.' + n.split('_')[1] for n in ListOfRandomSpecies]
stats = {}
types = [
    'high-fiber-diet',
    'unconstrained',
    'western-diet',
#    'average-european-diet',
 
]

def sensitivitySort(DietSensitivity):
    metnames=['']
    metsensitivity=[0]
    check=0
    for metabolite in DietSensitivity.keys():
        if check == 0:
            check+=1
            metnames[0]=metabolite
            metsensitivity[0]=DietSensitivity[metabolite]
        else:
            i=0
            while i<len(metnames):
                if DietSensitivity[metabolite] >  metsensitivity[i]:
                    metsensitivity[i+1:]=metsensitivity[i:]
                    metsensitivity[i]= DietSensitivity[metabolite]
                    metnames[i+1:]=metnames[i:]
                    metnames[i]=metabolite
                    break
                elif i==len(metnames)-1:
                    metsensitivity.append(DietSensitivity[metabolite])
                    metnames.append(metabolite)
                    break
                i+=1
    return metnames, metsensitivity

def getRelevantDictionaries(diet, species):
    model = cobra.io.read_sbml_model("./dfba/data/"+ diet +"/" + species + ".xml")
    AllExchanges = [r.id for r in model.exchanges]
    solution = model.optimize()
    igr = solution.objective_value
    DietSensitivity = {}
    originallb = {}
    for r in AllExchanges:
        if r not in DietSensitivity.keys():
            originallb[r] = model.reactions.get_by_id(r).lower_bound
            model.reactions.get_by_id(r).lower_bound = 0
            s = model.optimize()
            gr = s.objective_value
            DietSensitivity[r] = abs(igr-gr)/igr
            model.reactions.get_by_id(r).lower_bound = originallb[r] 
    return model, DietSensitivity, originallb

def getMinMetabolites(diet,species,SenThresh,Clusters=False):
    model, DietSensitivity, originallb = getRelevantDictionaries(diet, species)
    metnames, metsensitivity = sensitivitySort(DietSensitivity)
    s = model.optimize()
    gro = s.objective_value
    EssentialMetabolites = {}
    NonEssentialMetabolites = {}
    for i in range(len(metnames)-1,-1,-1):
#         s = model.optimize()
#         gri = s.objective_value
        model.reactions.get_by_id(metnames[i]).lower_bound = 0
        s = model.optimize()
        gr = s.objective_value
        Sens = abs(gro-gr)/gro
        if Sens > SenThresh: #if essential
            EssentialMetabolites[metnames[i]]=Sens
            model.reactions.get_by_id(metnames[i]).lower_bound = originallb[metnames[i]]
        elif originallb[metnames[i]] < 0:
            NonEssentialMetabolites[metnames[i]]=Sens
    if Clusters== False:
        return EssentialMetabolites, NonEssentialMetabolites
    else:
        return  model, DietSensitivity, originallb, EssentialMetabolites, NonEssentialMetabolites
    
def findGrowthRates(MinMetabolites, diet, species):
    model = cobra.io.read_sbml_model("./dfba/data/"+ diet +"/" + species + ".xml")
    AllExchanges = [r.id for r in model.exchanges]
    OptimalSolution = model.optimize()
    OptimalGrowth = OptimalSolution.objective_value
    for r in AllExchanges:
        if r not in MinMetabolites:
            model.reactions.get_by_id(r).lower_bound = 0.0
    MinSolution = model.optimize()
    MinMetaboliteGrowth = OptimalSolution.objective_value
    return OptimalGrowth, MinMetaboliteGrowth

def getExcretedMetabolites(model):
    AllExchanges = [r.id for r in model.exchanges]
    MinSolution = model.optimize()
    ExcretedMetabolites = []
    for r in AllExchanges:
        if MinSolution.fluxes[r] > 0:
            ExcretedMetabolites.append(r)
    return ExcretedMetabolites

def defineClusters(diet,species,SenThresh):
#     start=time.clock()
#     print('Clustering Now...'
    model, DietSensitivity, originallb, EssentialMetabolites, NonEssentialMetabolites = getMinMetabolites(diet,species,SenThresh,True)
    newclocktime = time.clock()
#     print('Time for MinMet =' + str(newclocktime-start))
    print(species + ' has ' + str(len(EssentialMetabolites)) + ' essential metabolites')
    print('\tand ' + str(len(NonEssentialMetabolites)) + ' non-essential metabolites')
    s = model.optimize()
    gro = s.objective_value
    Clusters = {}
    ExcretedMetabolites = getExcretedMetabolites(model)
    UniqueMetabolites = []
    for cluster in EssentialMetabolites:
        ClusterStorage = [cluster]
        model.reactions.get_by_id(cluster).lower_bound = 0
        s = model.optimize()
        kogr = s.objective_value
        ExcretionSame = 'Yes'
        NewExcretedMetabolites = ExcretedMetabolites
        for metabolite in NonEssentialMetabolites:
            model.reactions.get_by_id(metabolite).lower_bound = originallb[metabolite]        
            s = model.optimize()
            metgr = s.objective_value
            if (metgr-kogr)/(gro-kogr) >= SenThresh*0.99:
                ClusterStorage.append(metabolite)
                NewExcretedMetabolites = getExcretedMetabolites(model)
                ExcretedMetabolites, NonOverlap = getMetabolicOutputOverlap(ExcretedMetabolites,NewExcretedMetabolites)
                UniqueMetabolites = set(UniqueMetabolites) | NonOverlap
                CombinedExcretion = ExcretedMetabolites | UniqueMetabolites
                if len(list(CombinedExcretion)) != len(list(ExcretedMetabolites)) or  len(ExcretedMetabolites) != len(NewExcretedMetabolites):
                    ExcretionSame = 'No'
            model.reactions.get_by_id(metabolite).lower_bound = 0   
        Clusters[cluster] = {'ClusterMetabolites':ClusterStorage,'SharedMetabolites': ExcretedMetabolites, 'UniqueMetabolites':UniqueMetabolites, 'ExcretionTheSame': ExcretionSame}
        model.reactions.get_by_id(cluster).lower_bound = originallb[cluster]   
#     print('ClusterTime =' + str(time.clock()-newclocktime))
    return Clusters

def cleanupname(name):
    """
     The reaction names in the model files 
     don't have brackets or parentheses. I replaced
     those found in the mediaFluxes file.
     """
    name = name.replace('[', '__40__')#_LPAREN_')
    name = name.replace(']', '__41__')#'_RPAREN_')
    name = name.replace('(', '__40__')#'_LPAREN_')
    name = name.replace(')', '__41__')#'_RPAREN_')
    return name


def unsatisfiedClusters(Clusters,dietSet):
    A = set()
    B = set()
    for metabolite in dietSet:
        for clust in Clusters.keys():
            B.add(clust)
            if metabolite in Clusters[clust]['ClusterMetabolites']:
                A.add(clust)
    return B-A

def requiredNutrients(CommunitySpecies, CommunityType, dietFile, RemoveOutExchanges=False):
    AllEssentialMetabolites = set()
    dietSet = set()
    for i, row in dietFile.iterrows():
        N = cleanupname(row.Reaction)
        dietSet.add(N)
    for species in CommunitySpecies:
        print(species)
        model, DietSensitivity, originallb, EssentialMetabolites, NonEssentialMetabolites = getMinMetabolites(CommunityType,species,0.99,Clusters=True)
        excreted = set()
        if RemoveOutExchanges == True:
            for r in originallb.keys():
                if originallb[r] >=0:
                    excreted.add(r)
        AllEssentialMetabolites = (AllEssentialMetabolites | set(EssentialMetabolites))-excreted
    Overlap,ClusterSetMissed= getMetabolicOutputOverlap(dietSet,AllEssentialMetabolites)
    return ClusterSetMissed
  
    
    
def getMetabolicOutputOverlap(ClusterA, ClusterB):
    A = set(ClusterA)
    B = set(ClusterB)
    Overlap =  A & B
    Unique = (A-B) | (B-A)
    return Overlap, Unique


In [3]:
# for i in ListofRandomSpecies:
#     Communtiy = ListofRandomSpecies[i]
#     break
Community = ListOfRandomSpecies
dietFile = pd.read_csv('./dfba/data/diet-definitions/VMH_HighFiber.tsv', sep='\t')
ClustersMissed = requiredNutrients(Community,'average-european-diet',dietFile,False)
print(ClustersMissed)

Bacteroides_thetaiotaomicron_VPI_5482
Bacteroides_sp_2_2_4
Parabacteroides_johnsonii_DSM_18315
Prevotella_oralis_ATCC_33269
Eubacterium_eligens_ATCC_27750
Slackia_exigua_ATCC_700122
Dorea_longicatena_DSM_13814
Clostridium_bartlettii_DSM_16795
Streptococcus_sp_I_P16
Blautia_hydrogenotrophica_DSM_10507
Brevundimonas_bacteroides_DSM_4726
Clostridium_hylemonae_DSM_15053
Sutterella_wadsworthensis_3_1_45B
Enterobacteriaceae_bacterium_9_2_54FAA
Bacillus_megaterium_DSM319
Brachybacterium_paraconglomeratum_LC44
Neisseria_elongata_subsp_glycolytica_ATCC_29315
Rothia_aeria_F0474
Staphylococcus_hominis_subsp_hominis_C80
{'sink_gthrd__40__c__41__', 'EX_26dap_M__40__e__41__', 'EX_glyc__40__e__41__', 'EX_btn__40__e__41__', 'EX_arab_D__40__e__41__', 'EX_mnl__40__e__41__', 'EX_vitd3__40__e__41__', 'EX_avite1__40__e__41__', 'EX_thymd__40__e__41__', 'EX_urate__40__e__41__', 'EX_sbt_D__40__e__41__', 'EX_lgnc__40__e__41__', 'EX_ptdca__40__e__41__', 'EX_adpcbl__40__e__41__', 'EX_hdcea__40__e__41__', 'EX_cgl

In [ ]:
OverlapDict = {}
for sp in ListOfRandomSpecies:
    if 'Peptostreptococcus' in sp:
        continue
    else:
        Clusters = defineClusters('average-european-diet',sp,0.99)
        anything=0
        for A in Clusters.keys():
            if len(Clusters[A]['ClusterMetabolites']) > 1:
                anything += 1
                print('\tCluster ' + str(anything) + ' = ' + str(Clusters[A]['ClusterMetabolites']))
                print('\tMetabolic excretion the same?   ' + Clusters[A]['ExcretionTheSame'] )
                print('\tOverlapping excreted metabolites =' + str(len(Clusters[A]['SharedMetabolites'])))
                print('\tUnique excreted metabolites =' + str(len(Clusters[A]['UniqueMetabolites'])))
                OverlapDict[sp + '_Cluster_' + str(anything)] = len(Clusters[A]['SharedMetabolites'])/(len(Clusters[A]['SharedMetabolites'])+len(Clusters[A]['UniqueMetabolites']))
        if anything==0:
            print('\tand no clusters have size > 1')

# objects = ('Python', 'C++', 'Java', 'Perl', 'Scala', 'Lisp')
# y_pos = np.arange(len(objects))
# performance = [10,8,6,4,2,1]
# plt.bar(y_pos, performance, align='center', alpha=0.5)

# plotthis = {sp:stats[sp][t] for sp in stats.keys()}
# plt.bar([r + pos for r in range(len(plotthis))], plotthis.values(),width=0.15, align='center', label=t)

plotthis = {clust: OverlapDict[clust] for clust in OverlapDict.keys()}
plt.bar([r for r in range(len(plotthis))], plotthis.values(),width=0.15, align='center')
plt.xticks(range(len(plotthis)), list(plotthis.keys()), rotation=30)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import cobra
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm


ListOfRandomSpecies = [
'Bacteroides_thetaiotaomicron_VPI_5482',
'Bacteroides_sp_2_2_4',
'Parabacteroides_johnsonii_DSM_18315',
'Prevotella_oralis_ATCC_33269',
'Eubacterium_eligens_ATCC_27750',
'Slackia_exigua_ATCC_700122',
'Dorea_longicatena_DSM_13814',
'Clostridium_bartlettii_DSM_16795',
'Streptococcus_sp_I_P16',
'Blautia_hydrogenotrophica_DSM_10507',
'Brevundimonas_bacteroides_DSM_4726',
'Clostridium_hylemonae_DSM_15053',
'Sutterella_wadsworthensis_3_1_45B',
'Enterobacteriaceae_bacterium_9_2_54FAA',
'Bacillus_megaterium_DSM319',
#'Peptostreptococcus_stomatis_DSM_17678', # average-european-diet causes problems
'Brachybacterium_paraconglomeratum_LC44',
'Neisseria_elongata_subsp_glycolytica_ATCC_29315',
'Rothia_aeria_F0474',
'Staphylococcus_hominis_subsp_hominis_C80', 
]

betternames = [n.split('_')[0][0] + '.' + n.split('_')[1] for n in ListOfRandomSpecies]
stats = {}
types = [
    'high-fiber-diet',
   # 'unconstrained',
    'western-diet',
    'average-european-diet',
 
]
for t in types:
    print(t)
    for sp in ListOfRandomSpecies:
        if 'Peptostreptococcus' in sp and t=='average-european-diet':
            continue
        else:
            if sp not in stats.keys():
                stats[sp] = {}
            print(sp)
            EssentialMetabolites, NonEssentialMetabolites = getMinMetabolites(t,sp,0.75)
            OptimalGrowth, MinMetaboliteGrowth = findGrowthRates(EssentialMetabolites, sp)

            print("\tOptimal growth rate:" + str(OptimalGrowth))
            print("\tWith minimal nutrient set: " + str(MinMetaboliteGrowth))
            print("\tThe minimal set contains " + str(len(EssentialMetabolites)) + " metabolites")
    
            stats[sp][t]=len(EssentialMetabolites)
            

 

In [ ]:
#del stats['Peptostreptococcus_stomatis_DSM_17678']
for s in stats:
    if 'Peptostreptococcus' not in s:
        print(s + '\t' + str(stats[s][types[0]]) + '\t' +  str(stats[s][types[1]]) + '\t' +  str(stats[s][types[2]]))


pos = -0.2
for t in types:
    plotthis = {sp:stats[sp][t] for sp in stats.keys()}
    plt.bar([r + pos for r in range(len(plotthis))], plotthis.values(),width=0.15, align='center', label=t)
    pos +=0.2
plt.xticks(range(len(plotthis)), betternames,rotation=30)
plt.legend()
plt.show()
    